# COP export

This script reads and reorganises the raw ArchetypeBuildingModel coefficient of performance timeseries for Mopo WP5.

## Julia environment setup

In [ ]:
## Julia environment setup

using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using CSV
using DataFrames
using Dates
using Statistics

## Read and reorganise the raw ABM COP data.

In [ ]:
## Config for the raw data

years = [1995, 2008, 2009, 2012, 2015]
cop_paths = [
    year => "input-data/abm-raw-data/process_cops_$(year).csv"
    for year in years
];

In [ ]:
## Read and reorganise COP data

# Read and reorganise the data.
cop_cols = [:timestamp, :weather_year, :country, :category, :process, :value]
cop_data = DataFrame()
for (year, path) in cop_paths
    df = stack(DataFrame(CSV.File(path; header=[1,2])))
    s = split.(df[!, :variable], '_')
    df[!, :country] = String.(getindex.(s, 1))
    df[!, :category] = String.(getindex.(s, 2))
    df[!, :process] = String.(getindex.(s, 5))
    df[!, :distribution] = String.(get.(s, 6, "air"))
    df[!, :weather_year] .= year
    df[!, :process] = df[!, :process] .* '-' .* df[!, :distribution]
    rename!(
        df,
        :building_archetype_building_process => :timestamp,
    )
    append!(cop_data, df[!, cop_cols])
end

# Get rid of any potential sub-one COPs
cop_data.value = max.(cop_data.value, 1.0)
describe(cop_data)

## Export desired timeseries.

In [ ]:
## Export timeseries

# Export config
years = [1995, 2008, 2009, 2012, 2015]
dgts = 3 # Number of digits when rounding export
cat = "res" # Only export `res` COPs, there's no meaningful difference anyhow.
processes = Set(cop_data.process)
export_cols = [:timestamp, :country, :value]

# Export csvs
for pro in processes
    for year in years
        df = filter(
            r -> r.weather_year == year && r.category == cat && r.process == pro,
            cop_data
        )
        df = df[!, export_cols]
        df.value = round.(df.value; digits=dgts)
        df = unstack(df, :country, :value)
        CSV.write("output/cop-timeseries/COP_$(pro)_wy$(year).csv", df)
    end
end

## Plots to visually inspect the data

In [ ]:
## Plot diagnostics

using Plots

plot_year = 1995
plot_cols = [:timestamp, :process, :value]
cat = "res"
countries = Set(cop_data.country)
f = nothing # How many timesteps to plot, `nothing` plots all

for c in countries
    df = filter(
        r -> r.weather_year == plot_year && r.category == cat && r.country == c,
        cop_data
    )
    df = df[!, plot_cols]
    df = unstack(df, :process, :value)
    isnothing(f) ? nothing : df = first(df, f)
    plt = plot(
        df[!, 1],
        Matrix(df[!, 2:end]);
        title = "$(c)",
        labels = reshape(names(df)[2:end], 1, length(names(df)) - 1)
    )
    display(plt)
end